In [2]:
# !pip install --user -U nltk
import nltk
import pandas as pd
import requests
import urllib.request
import os, time
from bs4 import BeautifulSoup
import urllib

In [3]:
def retrieve_categories(base_url, category_head):
    
    ### Get all Microdata categories ###
    categories_url = base_url+category_head
    categories_response = requests.get(categories_url)

    categories_soup = BeautifulSoup(categories_response.text)
    categories_browse = categories_soup.findAll('a')
    
    categories_url_list = []
    categories_list = []
    for item in categories_browse:
        try:
            if category_head in item['href']:
                categories_url_list.append(base_url+item['href'])
                categories_list.append(item['href'].split('/')[-1])
        except KeyError:
            pass
    
    return categories_url_list, categories_list

In [4]:
def retrieve_variables(variables_url, variable_base_url):

    ### Crawl all variables names on the category web page ###

    variables_response = requests.get(variables_url)

    variables_soup = BeautifulSoup(variables_response.text)
    variables_browse = variables_soup.findAll('a')

    ### List all variables into a list ###
    variables_list = []
    for item in variables_browse:
        try:
            variable = item["data-id"][:-2]
            variables_list.append(variable)
        except KeyError:
            pass
            
    variable_url_list = []
    for item in variables_list:
        variable_url_list.append(variable_base_url+item)
            
    return variable_url_list, variables_list

In [5]:
def download_page(variable_url_list, base_url):
### Obtain Metadata phd file ###
    time.sleep(1)
    download_pages = []
    landing_pages = []

    for each_variable_url in variable_url_list:

        each_variable_response = requests.get(each_variable_url)

        each_variable_soup = BeautifulSoup(each_variable_response.text)
        each_variable_browse = each_variable_soup.findAll('a')

        for item in each_variable_browse:
            try:
                if ".pdf" in item["href"]:
                    download_pages.append(base_url+item["href"])
                    landing_pages.append(each_variable_url)
            except KeyError:
                pass
            except:
                print("This is not a KeyError!")
                
    return download_pages, landing_pages

In [ ]:
### Retrievel landing and downloading path ###
base_url = "https://www.cbs.nl"
category_head = "/nl-nl/onze-diensten/maatwerk-en-microdata/microdata-zelf-onderzoek-doen/catalogus-microdata/"
variable_base_url = "https://www.cbs.nl/nl-nl/onze-diensten/maatwerk-en-microdata/microdata-zelf-onderzoek-doen/microdatabestanden/"

# Retrieve categories #
categories_url_list, categories_list = retrieve_categories(base_url, category_head)
print("There are %s categories" %str(len(categories_url_list)))

overview_landingPage = dict()
overview_downloadPage = dict()
# Retrieve variables in each category #
for category_item in range(0, len(categories_url_list)):

    variable_url_list, variables_list = retrieve_variables(categories_url_list[category_item], variable_base_url)
    print("In category (%s), there are %s variables" %(categories_list[category_item], str(len(variable_url_list))))
#     overview.append(variables_list)
    download_pages, landing_pages = download_page(variable_url_list, base_url)
    overview_landingPage.update({categories_list[category_item]: landing_pages})
    overview_downloadPage.update({categories_list[category_item]: download_pages})
#     download_pdf(download_pages, categories_list[category_item])

    print("****** Retrieval of %s is done! ******" %(categories_list[category_item]))

In [10]:
topics_df_EN = pd.read_csv("CBSMicrodataInfoEN.csv")
topics_df_NL = pd.read_csv("CBSMicrodataInfoNL_2.csv")

variables_df_EN = pd.read_csv("CBSMicrodataVariablesEN.csv")
topics_df_EN['File_id'] = topics_df_NL['File_id'] 

In [ ]:
# topics_df_EN
downloadPage=[]
landingPage=[]
for i in range(0, len(topics_df_EN['File_id'])):
    cnt_pre = len(downloadPage)
    for c in overview_downloadPage.keys():
        for n in range(0, len(overview_downloadPage[c])):
            if topics_df_EN['File_id'][i] == overview_downloadPage[c][n].replace('?la=nl-nl','').split('/')[-1][:-4]:
                if topics_df_EN['Category'][i] == c:
                    downloadPage.append(overview_downloadPage[c][n])
                    landingPage.append(overview_landingPage[c][n])
                    
    cnt_post = len(downloadPage)
    if cnt_pre != cnt_post-1:
        print(topics_df_EN['File_id'][i])
        print(cnt_pre, cnt_post)
        downloadPage.append(None)
        landingPage.append(None)
    
topics_df_EN['Download_page'] = downloadPage
topics_df_EN['Landing_page'] = landingPage

In [34]:
topics_df_combined = pd.read_csv("CBSMicrodataInfo_combined_v2.csv")
# topics_df_combined_eywordv3 = topics_df_combined.drop(['Keyword_1', 'Keyword_2', 'Keyword_3', 'Keyword_4', 'Keyword_5',\
#                  'K_6', 'Keyword_7','Keyword_8', 'Keyword_9'], axis=1)
# topics_df_combined_v3

In [57]:
### Split DATE to YEAR MONTH DAY ###
Year = []
Month = []
Day = []
for i in topics_df_combined['Date_EN']:
    try:
        if int(i.split('-')[0])==0:
            Day.append(None)
        else:
            Day.append(i.split('-')[0])
        Month.append(i.split('-')[1])
        Year.append(i.split('-')[2])
    except:
        Day.append(None)
        Month.append(None)
        Year.append(None)
        
topics_df_combined['Year']=Year
topics_df_combined['Month']=Month
topics_df_combined['Day']=Day

topics_df_combined['Date_EN']

In [84]:
Keywords=[]
for sentence in topics_df_combined['Title_EN']:
    each_topic_keywords=[]
    try:
        tokens = nltk.word_tokenize(sentence)
        tagged = nltk.pos_tag(tokens)
    except:
        Keywords.append(None)
    else:
    
        for i in tagged:
            if i[1] == 'NN': #or i[1] == 'NNP':
                if i[0] not in each_topic_keywords:
                    each_topic_keywords.append(i[0])
                
        Keywords.append(each_topic_keywords)
### Collect all words ###
new_key=[]
for i in Keywords:
    try:
        for j in i:
            new_key.append(j)
    except:
        pass
len(new_key)

In [188]:
### Take words which appear more than twice ###
new_key_2=[]
for i in Keywords:
    each=[]
    try:
        for j in i:
            if Counter(new_key)[j] > 2:
                each.append(j)
#             else:
#                 each.append()
    except:
        each.append(None)
    
    new_key_2.append(each)

In [ ]:
key_df = pd.DataFrame.from_records(new_key_2)
key_df.columns = ['Keyword_1', 'Keyword_2', 'Keyword_3', 'Keyword_4', 'Keyword_5']
key_df

In [4]:
### Named Entity Recognition ###
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [168]:
notes_keywords = []
for i in topics_df_combined['Notes_EN']:
    each_note = []
    
    try:
        doc = nlp(i)
        for X in doc.ents:
            if X.label_ == 'ORG' and X.text != 'CBS':
                if X.text not in each_note:
                    each_note.append(X.text)
                
            elif X.label_ =='DATE':
                for item in X.text.split(' '):
                    if item.isdigit():
                        
                        if (int(item) > 1900 and int(item) < 2500) or (int(item) > 10000):
                            if item not in each_note:
                                each_note.append(item)


        notes_keywords.append(each_note)
    
    except:
        notes_keywords.append(None)

In [169]:
notes_keywords
### Collect all words ###
new_notes_keywords=[]
for i in notes_keywords:
    try:
        for j in i:
            new_notes_keywords.append(j)
    except:
        pass
len(new_notes_keywords)

2343

In [215]:
### Take words which appear more than twice ###
new_notes_keywords_2=[]
for i in notes_keywords:
    each=[]
    try:
        for j in i:
            if Counter(new_notes_keywords)[j] > 2:
                if j not in ["Research See","the Version History", "Remote Access", "\uf0b7",\
                            "Code_Listings","The Version History","RINPERSOON", "GBA"]:
                    if j == "the Production Statistics (PS" and "Production Statistics" not in each:
                        each.append("Production Statistics")
                    elif j == "SBS" and "the Structural Business Statistics" not in each:
                        each.append("the Structural Business Statistics")
                    else:
                        if j not in each:
                            each.append(j)
                    
              
    except:
        each.append(None)
    
    new_notes_keywords_2.append(each)

In [217]:
# Counter(new_notes_keywords).most_common()
# len(notes_keywords)

In [218]:
key_df_add = pd.DataFrame.from_records(new_notes_keywords_2)

In [248]:
new_key_df=[]
for i in range(0, len(key_df)):
    flag = False
    new_row = []
    for item in key_df.iloc[i]:
        if item:
            new_row.append(item)
        if not item:
            if not flag:
                flag=True
                for add_item in key_df_add.iloc[i]:
                    if add_item:
                        new_row.append('_'.join(word for word in add_item.split(' ')))
    new_key_df.append(new_row)

In [ ]:
new_key_df=pd.DataFrame.from_records(new_key_df)
new_key_df.columns = ['Keyword_1', 'Keyword_2', 'Keyword_3', 'Keyword_4', 'Keyword_5',\
                     'Keyword_6', 'Keyword_7', 'Keyword_8', 'Keyword_9', 'Keyword_10',\
                     'Keyword_11', 'Keyword_12' ,'Keyword_13', 'Keyword_14', 'Keyword_15', 'Keyword_16']
# new_key_df

In [242]:
for i in range(1,17):
    topics_df_combined_v3['Keyword_%s' %str(i)]=new_key_df['Keyword_%s' %str(i)]

In [245]:
topics_df_combined_v3.to_csv('CBSMicrodataInfo_combined_v3.csv',index=None)

In [22]:
brief_des=[]
for i in variables_df_new['Variable_def_EN']:
    try:
        brief_des.append(i.split('.')[0])
    except:
        brief_des.append(None)

In [24]:
Keywords=[]
for sentence in brief_des:
    each_topic_keywords=[]
    try:
        tokens = nltk.word_tokenize(sentence)
        tagged = nltk.pos_tag(tokens)
    except:
        Keywords.append(None)
    else:
    
        for i in tagged:
            if i[1] == 'NNS' or i[1] == 'NNP':
                each_topic_keywords.append(i[0])
        Keywords.append(each_topic_keywords)